In [8]:
import torch
import torch.nn as nn
import numpy as np
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

In [37]:
#Hyperparameters
num_epochs = 10
batch_size = 100
learning_rate = 0.001

In [10]:
train_set = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_set = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_set, batch_size=batch_size, shuffle=False)

In [38]:
classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')

In [39]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3) # Input channels = 1 , Output channels (The number of filter) = 6, Filter (Kernel) size = 5
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
model = ConvNet()

In [40]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [41]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

Epoch [1/10], Step [100/15000], Loss: 1.4102
Epoch [1/10], Step [200/15000], Loss: 0.3418
Epoch [1/10], Step [300/15000], Loss: 0.0410
Epoch [1/10], Step [400/15000], Loss: 0.9825
Epoch [1/10], Step [500/15000], Loss: 0.0432
Epoch [1/10], Step [600/15000], Loss: 0.1817
Epoch [1/10], Step [700/15000], Loss: 0.0820
Epoch [1/10], Step [800/15000], Loss: 0.5458
Epoch [1/10], Step [900/15000], Loss: 0.5983
Epoch [1/10], Step [1000/15000], Loss: 0.1976
Epoch [1/10], Step [1100/15000], Loss: 0.0526
Epoch [1/10], Step [1200/15000], Loss: 0.0553
Epoch [1/10], Step [1300/15000], Loss: 0.0269
Epoch [1/10], Step [1400/15000], Loss: 0.2759
Epoch [1/10], Step [1500/15000], Loss: 0.2506
Epoch [1/10], Step [1600/15000], Loss: 0.1331
Epoch [1/10], Step [1700/15000], Loss: 0.3510
Epoch [1/10], Step [1800/15000], Loss: 0.0203
Epoch [1/10], Step [1900/15000], Loss: 0.0052
Epoch [1/10], Step [2000/15000], Loss: 0.2027
Epoch [1/10], Step [2100/15000], Loss: 0.0061
Epoch [1/10], Step [2200/15000], Loss: 1.17

In [42]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

# Evaluation
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    all_preds = []
    all_labels = []

    for images, labels in test_loader:
        #k reshape ảnh khi dùng CNN
        labels = labels

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        # Accuracy
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

        # Lưu lại để tính precision/recall/F1
        #extend để thêm từng phần tử vào danh sách khác với append thêm tất cả mảng 1 lần
        all_preds.extend(predicted.cpu().numpy()) 
        all_labels.extend(labels.cpu().numpy())

# Accuracy
acc = 100.0 * n_correct / n_samples
print(f'\n✅ Accuracy of the model on the 10000 test images: {acc:.2f} %')

# Precision / Recall / F1 (macro: trung bình đều giữa các lớp)
precision = precision_score(all_labels, all_preds, average='macro')
recall = recall_score(all_labels, all_preds, average='macro')
f1 = f1_score(all_labels, all_preds, average='macro')

print(f'🎯 Precision: {precision:.4f}')
print(f'🔁 Recall:    {recall:.4f}')
print(f'📏 F1 Score:  {f1:.4f}')

# Báo cáo chi tiết theo từng lớp
print("\n📊 Classification report:")
print(classification_report(all_labels, all_preds))



✅ Accuracy of the model on the 10000 test images: 98.60 %
🎯 Precision: 0.9860
🔁 Recall:    0.9859
📏 F1 Score:  0.9859

📊 Classification report:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       980
           1       0.99      0.99      0.99      1135
           2       0.98      0.99      0.99      1032
           3       0.99      0.99      0.99      1010
           4       0.98      0.99      0.99       982
           5       0.98      0.98      0.98       892
           6       0.99      0.98      0.98       958
           7       0.98      0.99      0.99      1028
           8       0.98      1.00      0.99       974
           9       0.99      0.96      0.97      1009

    accuracy                           0.99     10000
   macro avg       0.99      0.99      0.99     10000
weighted avg       0.99      0.99      0.99     10000

